# Create and use a Supervisor Agent to orchestrate multiple sub-agents and a knowledge base



The Supervisor agent plays a crucial role in multi-agent collaboration systems. It acts as a central coordinator, overseeing the activities of other specialized agents and ensuring cohesive, goal-oriented outcomes. The Supervisor's primary function is to manage task allocation, monitor progress, and integrate results from various agents. This hierarchical structure enhances efficiency, reduces conflicts, and maintains focus on the overall objective, ultimately leading to more effective problem-solving and decision-making in complex scenarios.


In this notebook, we will 
* Demonstrate orchestration of our mortgage sub-agents with a supervisor agent. 
* Create a guardrail that will act to regulate the types of inputs and outputs that are MAC can process
* Test our supervisor agent
* Measure the latency of our supervisor/sub-agent system


First step is to install the pre-requisites packages. NOTE: You only need to do this is this is the first notebook you are running. 

In [ ]:
# Only uncomment out and install requirements if this is the first notebook you are running
# !pip install --upgrade -q -r requirements.txt
# !pip install --upgrade -q boto3 botocore awscli 

In [ ]:
import boto3
print(f'boto3 version: {boto3.__version__}')
import logging
import time
import uuid

%load_ext autoreload
%autoreload 2

from knowledge_base import BedrockKnowledgeBase
from agent import AgentsForAmazonBedrock

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bedrock_client = boto3.client('bedrock')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
bucket_name = f'mac-workshop-{suffix}'
agent_foundation_models = [
    # "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0", 
    ]

In [ ]:
agents = AgentsForAmazonBedrock()

In [ ]:
from knowledge_base import BedrockKnowledgeBaseHelper
helper = BedrockKnowledgeBaseHelper()
kb_id = helper.get_kb()
kb_arn = f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"

print(kb_id)
print(kb_arn)

## 1. Make a Supervisor Agent on top of existing agents

#### Review Lambda implementation for Supervisor agent
Take a look at this reusable Lambda function which implements an Agent Action Group for a supervisor agent.
Based on an environment variable provided to the Lambda, the function knows which sub-agents it supports.
The signature to each agent is identical. Thus, dispatching is generic and invocation is as well.

In [ ]:
!pygmentize supervisor_agent_function.py

In [ ]:
sub_agent_names = ["existing_mortgage_agent", "mortgage_application_agent"]

In [ ]:
supervisor_agent_name = "mortgage_supervisor_agent"

In [ ]:
kb_arn 

In [ ]:
function_defs, supervisor_agent_arn = \
    agents.create_supervisor_agent(supervisor_agent_name, 
                                    sub_agent_names,
                                    agent_foundation_models,
                                    kb_arn,
                                    "Useful for answering questions about mortgage refinancing and for questions comparing various mortgage types")

In [ ]:
supervisor_agent_id = supervisor_agent_arn.split('/')[1]
supervisor_agent_id
%store supervisor_agent_id

Try one simple invoke to be sure the supervisor agent is working.

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
print(agents.invoke("my customer id is 8953. when is my payment due?", 
                    supervisor_agent_id, session_id=session_id))

## 2. Test the Supervisor Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks. You can invoke your agent with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API. Here we use the agents wrapper class instead.

In [ ]:
%%time
print(agents.invoke("my id is 8953. what is my balance?", 
                supervisor_agent_id, session_id=session_id, enable_trace=False))

In [ ]:
%%time
print(agents.invoke("nice. what's my maturity date?", supervisor_agent_id, session_id=session_id))

In [ ]:
%%time
print(agents.invoke("what docs do I need for my application?", 
                    supervisor_agent_id, session_id=session_id))

In [ ]:
%%time
print(agents.invoke("what docs have I already provided you for my application?", 
            supervisor_agent_id, session_id=session_id, enable_trace=False))

In [ ]:
%%time
print(agents.invoke("compare and contrast 15-year and 30-year mortgage types", 
                    supervisor_agent_id, session_id=session_id))

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "my customer ID is 8953. what is my mortgage balance and when is my next payment due?"
response = agents.invoke(query, supervisor_agent_id, session_id=session_id)
print(response)

## 3. Use session attributes to provide context to the Supervisor Agent

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [`promptSessionAttributes`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')

session_state = {
    "promptSessionAttributes": {
        "customer_ID": "498",
        "today": today
    }
}
session_state

In [ ]:
%%time
query = "what docs do I still owe you?"
print(agents.invoke(query, supervisor_agent_id, 
                session_id=session_id, session_state=session_state))

In [ ]:
%%time
query = "how many years until my maturity date?"
print(agents.invoke(query, supervisor_agent_id, 
        session_id=session_id, session_state=session_state, enable_trace=False))

## 4. Control the tone of the supervisor, independent of the sub-agents

In [ ]:
current_instructions = agents.get_agent_instructions_by_name(supervisor_agent_name)
current_instructions

In [ ]:
current_instructions += """
 The style and tone of your response should be that of a casual and friendly conversation 
on social media or a text stream. Add some humor, and use texting shorthand like lol."""
updated_details = agents.update_agent(supervisor_agent_name, new_instructions=current_instructions)
updated_details

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "hey bro, i'm customer 1234. when's my next pmt?"
print(agents.invoke(query, supervisor_agent_id, session_id=session_id, session_state=session_state, enable_trace=False))

In [ ]:
%%time
query = "amazing, dude. can u help me raise some cash for that? jk"
print(agents.invoke(query, supervisor_agent_id, session_id=session_id))

## 5. Try out a guardrail

In [ ]:
try:
    response = bedrock_client.create_guardrail(
        name='MortgageAssistantGuardrail',
        description='Guardrail for supervisor bot to block any non-mortgage related topics',
        topicPolicyConfig={
            'topicsConfig': [
                {
                    'name': 'Non-Mortgage',
                    'definition': 'Non-Mortgage refers to any topic outside of a mortgage and payment related questions. This could potentially include financial related topics such as investment advice, banking advice, etc.',
                    'examples': [
                        'Should I buy gold?',
                        'Is investing in stocks better than bonds?',
                        'When is it a good idea to invest in gold?',
                        'Should I cash out my house and buy a bunch of stocks?',
                    ],
                    'type': 'DENY'
                },
            ]
        },
        blockedInputMessaging='Sorry, your query violates our usage policies. We are not allowed to discuss non-mortgage related questions. To discuss the best investment advice for your current situation, please contact us on (XXX) XXX-XXXX and we will be happy to support you.',
        blockedOutputsMessaging='Sorry, I am unable to reply. Please contact us on (XXX) XXX-XXXX and we will be happy to support you.',
    )
except:
    response = bedrock_client.list_guardrails(
        maxResults=123,
    )
    for guardrail in response.get('guardrails', []):
        if guardrail.get('name') == 'MortgageAssistantGuardrail':
            response = guardrail
    print(response)
    bedrock_client.delete_guardrail(guardrailIdentifier=response.get("id"))
    time.sleep(30)
    response = bedrock_client.create_guardrail(
    name='MortgageAssistantGuardrail',
    description='Guardrail for supervisor bot to block any non-mortgage related topics',
    topicPolicyConfig={
            'topicsConfig': [
                {
                    'name': 'Non-Mortgage',
                    'definition': 'Non-Mortgage refers to any topic outside of a mortgage and payment related question. This could potentially include financial related topics such as investment advice, banking advice, etc.',
                    'examples': [
                        'Should I buy gold?',
                        'Is investing in stocks better than bonds?',
                        'When is it a good idea to invest in gold?',
                        'Should I cash out my house and buy a bunch of stocks?',
                    ],
                    'type': 'DENY'
                },
            ]
        },
        blockedInputMessaging='Sorry, your query violates our usage policies. We are not allowed to discuss non-mortgage related questions. To discuss the best investment advice for your current situation, please contact us on (XXX) XXX-XXXX and we will be happy to support you.',
        blockedOutputsMessaging='Sorry, I am unable to reply. Please contact us on (XXX) XXX-XXXX and we will be happy to support you.',
)
print(response)

guardrail_id = response['guardrailId']
guardrail_version = response['version']
updated_details = agents.update_agent(supervisor_agent_name, guardrail_id=guardrail_id)
time.sleep(10)


In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "hey bro, my customer ID is 123456. when's my next mortgage pmt?"
print(agents.invoke(query, supervisor_agent_id, session_id=session_id, enable_trace=False))

In [ ]:
%%time
query = "hey I've been reading alot about ETFs. Would you suggest I sell my house and move all the funds to ETFs?"
print(agents.invoke(query, supervisor_agent_id, session_id=session_id, enable_trace=False))

In [ ]:
%%time
query = "thanks. do you like the Boston Celtics? time for a new championship banner this year."
response = agents.invoke(query, supervisor_agent_id, 
                        session_id="200", enable_trace=False)
print(response)

# Sorry, the model cannot discuss basketball.

In [ ]:
updated_details = agents.update_agent(supervisor_agent_name, guardrail_id=None)

## 6. Quick performance test

Performance is a consideration when deciding on using a supervisor agent, intent classification, etc. In the below code the supervisor agent will be invoked multiple times to obtain averages for latency for each response to provide you an idea of it's overall impact to performance. Latency will vary based on model types you select for each agent and the integrations you utilize within your AWS Lambda functions.

In [ ]:
import uuid 
import time
import numpy as np

def query_loop_by_supervisor(query, agent_id, num_invokes):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()
        resp = agents.invoke(query, agent_id, session_id=_session_id)
        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nInvoked agent by supervisor {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [ ]:
query_loop_by_supervisor("I am customer id 999. when does my mortgage mature?", 
                            supervisor_agent_id, 25)

## 7. Clean-up 

**NOTE: Do not run these cells if you are proceeding to other notebooks**

In [ ]:
supervisor_agent_name = "mortgage_supervisor_agent"
agents.delete_lambda(f"{supervisor_agent_name}_lambda")
agents.delete_agent(supervisor_agent_name)

In [ ]:
response = bedrock_client.list_guardrails(
    maxResults=123,
)
for guardrail in response.get('guardrails', []):
    if guardrail.get('name') == 'MortgageAssistantGuardrail':
        response = guardrail
print(response)
bedrock_client.delete_guardrail(guardrailIdentifier=response.get("id"))

Only clean up the sub-agents if you are not moving on to subsequent notebooks.

In [ ]:
# agents.delete_lambda("existing_mortgage_ag")
# agents.delete_agent("existing_mortgage_agent")

# agents.delete_lambda("mortgage_application_ag")
# agents.delete_agent("mortgage_application_agent")